In [1]:
import geopandas as gpd
import pandas as pd
import subprocess

comercial_df = gpd.read_file('./data/LOTES/COMERCIAL/SHP/lotes.shp')
servicos_df = gpd.read_file('./data/LOTES/SERVIÇOS/SHP/lotes.shp')

In [2]:
def to_kml(df, output):
    command = "ogr2ogr -f KML {0}.kml {0}.json ".format(output)
    with open('{0}.json'.format(output), 'w') as f:
        f.write(df.to_crs(epsg=4326).to_json())
        
    subprocess.call(command,shell=True)
    
def export(df, output):
    kml_output = './OUTPUT/KML/' + output 
    csv_output = './OUTPUT/CSV/' + output + '.csv'
    shp_output = './OUTPUT/SHP/' + output
    
    to_kml(df, kml_output)
    df.to_file(shp_output)
    without_geometry = df.loc[:, df.columns != 'geometry']
    without_geometry.to_csv(csv_output)

In [3]:
def classify_comercio(row, column, dic):
    string = row[column]
    if string in dic['alimentacao']:
        return 1
    if string in dic['genAlimeticios']:
        return 2 
    if string in dic['artigosDoLar']:
        return 3 
    if string in dic['artigosDiversos']:
        return 4
    if string in dic['conveniencia']:
        return 5  
    if string in dic['vestuario']:
        return 6  
    if string in dic['construcao']:
        return 7
    if string in dic['automotivo']:
        return 8


def classify_servicos(row, column, dic):
    string = row[column]
    if string in dic['servicosAutomotivos']:
        return 8
    if string in dic['entretenimento']:
        return 9
    if string in dic['servicosDiversos']:
        return 10  
    if string in dic['servicosPessoais']:
        return 11
    if string in dic['servicosMedicos']:
        return 12
    
def classify_comercio_desc(row, column, dic):
    string = row[column]
    if string in dic['alimentacao']:
        return 'Alimentação'
    if string in dic['genAlimeticios']:
        return 'Gêneros Alimentícios'
    if string in dic['artigosDoLar']:
        return 'Artigos do Lar'
    if string in dic['artigosDiversos']:
        return 'Artigos Diversos'
    if string in dic['conveniencia']:
        return 'Conveniência'  
    if string in dic['vestuario']:
        return 'Vestuários'  
    if string in dic['construcao']:
        return 'Construção'
    if string in dic['automotivo']:
        return 'Automotivo'


def classify_servicos_desc(row, column, dic):
    string = row[column]
    if string in dic['servicosAutomotivos']:
        return 'Automotivo'
    if string in dic['entretenimento']:
        return 'Entretenimento'
    if string in dic['servicosDiversos']:
        return 'Serviços Diversos'  
    if string in dic['servicosPessoais']:
        return 'Serviços Pessoais'
    if string in dic['servicosMedicos']:
        return 'Serviços Médicos'

In [4]:
servicos_dict = {}
servicos_dict['servicosAutomotivos'] =  [ 'LOCACAO DE VEICULOS,EMBARCACOES OU AERONAVES',
'OFICINA MECANICAS OU LANTERNAGENS E PINTURA'
]

servicos_dict['servicosPessoais'] = [ 'HIGIENE E ESTETICA PESSOAL',
 'ESCOLAS DE ESPORTES OU LUTAS',
 'PREST.SERV-HIG.PES.E COND.FIS.',
 'ACADEMIA DE GINASTICA OU MUSCULACAO'
]

servicos_dict['entretenimento'] = [  'LOTERIAS,JOGOS DE APOSTAS OU BINGOS',
 'RECREACAO INFANTIL OU PARQUE DE DIVERSAO',
 'LOCACAO DE FILMES,DISCO,JOGOS OU CDS'

]

servicos_dict['servicosMedicos'] = [  'FISIOTERAPIA E RECUPERACAO',
 'PREST.SERV-SAUDE',
 'CONSULTORIO MEDICO',
 'CLINICA MEDICA',
 'CLINICA ODONTOLOGICA',
 'CLINICA DE PSICOLOGIA',
 'RADIOLOGIA',
 'FONOAUDIOLOGIA',
 'MEDICO',
]

servicos_dict['servicosDiversos'] = [  'AGENCIA DE TURISMO,VIAGENS OU VENDAS DE PASSAGENS',
'ESTUDIO FOTOGRAFICO',
 'CONSTRUCAO CIVIL E ATIVIDADES AFINS',
 'PREST.SERV-EDUCACAO',
 'ASSISTENCIA SOCIAL,RELIGIOSA OU BENEFICENTE',
 'ENSINO DE 1 GRAU,2 GRAU OU 3 GRAU',
 'SERVICOS DE TELECOMUNICACOES',
 'CORRETAGEM DE PLANOS DE SAUDE OU SEGUROS',
 'PREST.SERV-ARQ.CONST. CIVIL',
 'CORRETAGEM DE BENS MOVEIS',
 'INSTITUICOES FINANCEIRAS GERAL',
 'SERVICOS DE BUFFET',
 'PREST.SERV-FIN.SEC.E CAPITAL.',
 'LOCACAO DE MAQUINAS,APARELHOS OU EQUIPAMENTOS',
 'CURSOS DE IDIOMAS',
 'CORRETAGEM DE BENS IMOVEIS',
 'MATERNAL INFANTIL OU CRECHES',
 'AUTO-ESCOLA',
 'ENSINO TECNICO,INDUSTRIAL OU COMERCIAL',
 'COPIAS EM GERAL',
 'OFICINA DE FERREIRO - SERRALHERIA',
]

In [5]:
comercial_dict = {}
comercial_dict['genAlimeticios'] =  ['SUPERMERCADO,OU EMPORIO E MERCEARIAS',
'SUPERMERCADOS',
'PRODUTOS ALIMENTICIOS OU BEBIDAS',
'ALIMENTOS, BEBIDAS EM GERAL',
'FRIGORIFICOS, CARNES E AVES',
'SUPERMERCADO, OU EMPORIO E MERCEARIA SEM LIXO ORGANICO',
'ESTIVAS E CEREAIS']

comercial_dict['conveniencia'] = [ 'MEDICAMENTOS,COSMETICOS OU PERFUMES E CONGENERES',
'PAPELARIA,ARMARINHOS,BOUTIQUE OU PRESENTES E BOMBONIERE',
'MEDICAMENTOS,COSMETICOS OU PERFUMES',
'ARTIGOS DE TABACARIA',
'FLORICULTURA',
'CINE,FOTO OU SOM',
'PRODUTOS DE PAPELARIA',
'LIVROS JORNAIS OU REVISTAS',
'MAGAZINE',
'LOJAS DE DEPARTAMENTOS'
]

comercial_dict['vestuario'] = [ 'JOIAS,RELOGIOS,OTICA OU OBJETOS DE ARTE',
'TECIDOS,CONFECCOES,CALCADOS OU ESPORTES',
'CONFECCOES,CALCADOS BIJUTERIAS E CONGENERES',
'ARTIGOS DE COUROS E TANANTES'
]

comercial_dict['artigosDoLar'] = [ 'ELETRO ELETRONICOS OU INFORMATICA',
'MOVEIS EM GERAL',
'ARTIGOS DE ESPUMA E COLCHOES',
'ARTIGOS PARA DECORACAO,VIDROS E CONGENERES',
'PRODUTOS PLASTICOS E DESCARTAVEIS',
'ARTIGOS PLASTICOS E DESCARTAVEIS',
'ART. DE HIGIENE LIMPEZA EM GERAL',
'ANIMAIS E PRODUTOS VETERINARIOS'
]

comercial_dict['artigosDiversos'] = ['COMERCIO A VAREJO',
'ARTIGOS DE ARTESANATO',
'COMERCIAL -COMERCIO VAREJISTA',
'COMERCIO FECHADO',
'ARTIGOS MEDICOS,HOSPITALARES OU ODONTOLOGICO',
'MAQUINAS,  EQUIPAMENTOS E MATERIA PRIMA PARA PANIFICACAO',
'ARTIGOS FUNERARIOS',
'ARTIGOS AGROPECUARIOS',
'PRODUTOS QUIMICOS',
'MAQUINAS, APARELHOS E EQUIP. OU PECAS',
'ARTIGOS PARA CACA E PESCA E CONGEN  ERES',
'PRODUTOS ALIMENTICIOS PARA ANIMAIS',
'PRODUTOS MUSICAIS E INSTRUMENTOS',
'ARTIGOS RELIGIOSOS EM GERAL',
'EXPORTACAO E IMPORTACAO',
'PRODUTOS DE AGROPECUARIA',
'ARTIGOS PIROTECNICOS',
]

comercial_dict['construcao'] = ['MATERIAL DE CONSTRUCAO,MADEIRAS OU PEDRAS',
'EQUIPAMENTOS,APARELHOS,FERRAMENTAS OU PECAS',
'FERRAGENS,MATERIAIS ELETRICOS,HIDRAULICOS E CONGENERES',
'ADUBOS,TINTAS OU VERNIZES E CONGENERES',
'CASAS DE FERRAGENS'
]

comercial_dict['alimentacao'] = ['BAR, RESTAURANTE, LANCHONETE',
'LANCHONETE',
'SORVETERIA'
]

comercial_dict['automotivo'] = [ 'CONCESSIONARIAS DE VEICULOS E AUTO PECAS',
'COMBUSTIVEIS,LUBRIFICANTES OU GLP',
'VEICULOS E MAQUINAS PESADAS',
'PNEUS E ARTEFATOS DE BORRACHA',
'BICICLETAS, MOTOS E PECAS'
]

In [6]:
comercial_df['classificacao'] = comercial_df.apply(classify_comercio, axis=1, args=('Atividade', comercial_dict))
servicos_df['classificacao'] = servicos_df.apply(classify_servicos, axis=1, args=('Atividade', servicos_dict))
comercial_df['classificacao_desc'] = comercial_df.apply(classify_comercio_desc, axis=1, args=('Atividade', comercial_dict))
servicos_df['classificacao_desc'] = servicos_df.apply(classify_servicos_desc, axis=1, args=('Atividade', servicos_dict))

In [7]:
lotes_df = comercial_df.append(servicos_df, ignore_index=True)

In [8]:
recorte_maior = gpd.read_file('./data/BASES/limite_recorte/limite_recorte_maior.shp')
lotes_recorte =  gpd.overlay(lotes_df, recorte_maior, how='intersection')

In [9]:
bairros = gpd.read_file('./data/BASES/bairros/bairros.shp')
bairros_interesse = ['Torre', 'Tambauzinho', 'Expedicionários', 'Miramar', 'Pedro Gondim', 'Estados']
bairros_recorte = bairros.loc[bairros['NM_BAIRRO'].isin(bairros_interesse)]
bairros_recorte.crs

<Projected CRS: EPSG:3857>
Name: WGS 84 / Pseudo-Mercator
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: World - 85°S to 85°N
- bounds: (-180.0, -85.06, 180.0, 85.06)
Coordinate Operation:
- name: Popular Visualisation Pseudo-Mercator
- method: Popular Visualisation Pseudo Mercator
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [10]:
lotes_df = lotes_df.to_crs('epsg:3857')

In [13]:
lotes_with_bairros = gpd.sjoin(lotes_df, bairros_recorte, how="inner", op='intersects')

lotes_with_bairros['Area_Total'] = pd.to_numeric(lotes_with_bairros['Area_Total'])
lotes_with_bairros['Area_Terre'] = pd.to_numeric(lotes_with_bairros['Area_Terre'])
lotes_with_bairros['Area_Predi'] = pd.to_numeric(lotes_with_bairros['Area_Predi'])

grouped_lotes = lotes_with_bairros.groupby(['NM_BAIRRO', 'classificacao_desc']).agg({'classificacao_desc': 'count', 
                                                        'Area_Terre': 'sum',
                                                        'Area_Predi': 'sum',
                                                        'Area_Total': 'sum'
                                                       }).rename(columns={'classificacao_desc':'n_empreeendimentos'})


grouped_lotes = grouped_lotes.unstack().transpose()

grouped_lotes['total'] = grouped_lotes.sum(axis=1)
grouped_lotes.to_csv('./OUTPUT/CSV/lotes_por_bairro.csv',  decimal='.', sep=',', float_format='%.0f')

NM_BAIRRO                                  Estados  Expedicionários   Miramar  \
                   classificacao_desc                                           
n_empreeendimentos Alimentação               22.00             6.00     28.00   
                   Artigos Diversos          70.00            25.00     42.00   
                   Artigos do Lar            19.00             6.00     13.00   
                   Automotivo                13.00             2.00     14.00   
                   Construção                 9.00             3.00      8.00   
                   Conveniência              22.00             5.00     12.00   
                   Entretenimento             1.00              NaN       NaN   
                   Gêneros Alimentícios      13.00            10.00     21.00   
                   Serviços Diversos         93.00            22.00     61.00   
                   Serviços Médicos          30.00            10.00     27.00   
                   Serviços Pessoais         12.00             5.00     14.00   
                   Vestuários                26.00             3.00     11.00   
Area_Terre         Alimentação            13111.90          2816.50  25188.50   
                   Artigos Diversos      191896.76         15247.00  20936.10   
                   Artigos do Lar          9354.56          3401.50   5293.42   
                   Automotivo             15810.35          2595.00  11328.07   
                   Construção              7943.30          1275.60   3023.50   
                   Conveniência           16833.97          1382.50   5768.30   
                   Entretenimento           635.00              NaN       NaN   
                   Gêneros Alimentícios    8814.40          3571.18  10385.18   
                   Serviços Diversos      77865.35         18195.68  42012.42   
                   Serviços Médicos       24450.34          5814.95  13865.78   
                   Serviços Pessoais       6022.80          3578.00   6721.12   
                   Vestuários             14603.25          2365.50   5255.50   
Area_Predi         Alimentação             5904.75          1121.84  11480.21   
                   Artigos Diversos       40674.46          7753.32  10184.20   
                   Artigos do Lar          4539.10          1660.23   2247.43   
                   Automotivo              8357.99           298.21   4473.33   
                   Construção              4122.01           674.88   1340.24   
                   Conveniência            7894.91           704.50   4410.90   
                   Entretenimento           233.00              NaN       NaN   
                   Gêneros Alimentícios    5264.91           927.64   4173.23   
                   Serviços Diversos      50957.28          5893.30  20060.71   
                   Serviços Médicos       13520.42          3131.77   7076.34   
                   Serviços Pessoais       2910.91          2242.75   2536.28   
                   Vestuários              4703.23           625.63   2152.54   
Area_Total         Alimentação             6873.97          1146.84  14269.63   
                   Artigos Diversos       56791.83          8765.20  34910.52   
                   Artigos do Lar          6716.60          2461.23   2660.65   
                   Automotivo              9243.47           993.71   5643.33   
                   Construção              4139.76           674.88   1990.08   
                   Conveniência            9701.84           793.80   4982.01   
                   Entretenimento           233.00              NaN       NaN   
                   Gêneros Alimentícios    5893.24          3419.44   4388.82   
                   Serviços Diversos      78954.74         23394.85  38563.78   
                   Serviços Médicos       14384.97          4434.59   8570.61   
                   Serviços Pessoais       2960.51          2678.75   2976.47   
                   Vestuários  

In [13]:
export(lotes_df, 'lotes_varejo')
export(lotes_recorte, 'lotes_varejo_recorte')

ValueError: Cannot write empty DataFrame to file.

In [ ]:
#EXPORT WITH DESCRIPTION AS CLASSIFICACAO, ONLY ROWS IN VAREJO

def export_classificacao(df, output):
    df['classificacao'] = df['classificacao_desc']
    df = df.drop('classificacao_desc', axis=1)
    df = df.dropna(axis=0, how='any', subset=['classificacao'])
    export(df, output)
    
export_classificacao(lotes_df, 'lotes_varejo')
export_classificacao(lotes_recorte, 'lotes_varejo_recorte')